In [1]:
import pylangacq as pla

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
import glob

In [5]:
import os

In [6]:
pla.__version__

'0.11.0'

In [7]:
# data = pla.read_chat('./Pitt/*.cha')

Data grab

In [8]:
cookie_data = pla.read_chat('./Pitt/*/cookie/*.cha')

In [9]:
sentence_data = pla.read_chat('./Pitt/*/sentence/*.cha')

In [10]:
sentence_fnames = glob.glob(os.getcwd()+'/Pitt/*/sentence/*.cha')

In [11]:
sentence_fnames = sorted(sentence_fnames)

In [12]:
sentence_fnames[0]

'/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/001-0.cha'

In [13]:
# Control, Dementia
cookie_fnames = glob.glob(os.getcwd()+'/Pitt/*/cookie/*.cha')

In [14]:
cookie_fnames = sorted(cookie_fnames)

In [15]:
len(sentence_fnames+cookie_fnames)

792

In [16]:
# for random batching/data grabbing 
fnames_dict = {i:fname for (i,fname) in enumerate(sentence_fnames+cookie_fnames)}

Data exploration

In [17]:
len(cookie_data)

552

In [18]:
len(sentence_data)

240

In [19]:
cookie_data.number_of_utterances('PAR')

7054

Targets (MMSE scores)

In [20]:
# fill missing mmse with group's mean mmse
def get_targets(dataset,filenames):
    
    targets = []
    groups = {}
    
#     for fname in sorted(filenames):
    for fname in filenames:
        # PAR MMSE
#         print(fname)
        target = dataset.headers()[fname]['Participants']['PAR']['education']
        group = dataset.headers()[fname]['Participants']['PAR']['group']
        
        # to be replaced with group mean
        if target is '':
            targets.append(group)
            
        else:
            targets.append(int(target))
            if group in groups.keys():
                groups[group].append(int(target))
            else:
                groups[group] = [int(target)]
                
#     print('targets',targets)            
#     print('groups',groups)
    
    for g in groups.keys():
        m = np.mean(groups[g])
        groups[g] = int(m) if m % 1 > 5 else int(m) + 1
    
    #TODO replace with tensor
    targetseries = pd.Series(targets)
    
#     print('groups_mean',groups)
    
    for g in groups.keys():
        targetseries[[i for i,x in enumerate(targets) if x==g]] = groups[g]

    return targetseries

In [21]:
sentence_data_targets = get_targets(sentence_data,sentence_fnames)

In [22]:
cookie_data_targets = get_targets(cookie_data,cookie_fnames)

In [23]:
len(sentence_data_targets) == len(sentence_data)

True

In [24]:
len(cookie_data_targets) == len(cookie_data)

True

BUILD VOCAB

In [25]:
vocabset = set()

In [26]:
len(cookie_fnames)

552

In [27]:
for corpus,data in [(cookie_fnames,cookie_data),(sentence_fnames,sentence_data)]: 
    for i in range(len(corpus)):
        tokens = set([tokenraw for sent in data.tagged_sents(participant='PAR',by_files=True)[corpus[i]] for (tokenraw,pos,tokenstem,dependency) in sent])
        vocabset.update(tokens)

In [113]:
corpus = cookie_fnames
data = cookie_data

In [114]:
i

239

In [115]:
corpus[i]

'/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/688-0.cha'

In [164]:
for corpus,data in [(cookie_fnames,cookie_data),(sentence_fnames,sentence_data)]: 
    for i in range(len(corpus)):
        tokens = set([tokenraw for sent in data.tagged_sents(participant='PAR',by_files=True)[corpus[i]] for (tokenraw,pos,tokenstem,dependency) in sent])
        vocabset.update(tokens)
        
        break
        
    break

In [168]:
get_glovembs(tokens)['the'].shape

(300,)

In [171]:
import torch

In [172]:
get_glovembs(tokens)

{'the': tensor([-2.0838e-01, -1.4932e-01, -1.7528e-02, -2.8432e-02, -6.0104e-02,
         -2.6460e-01, -4.1445e+00,  6.2932e-01,  3.3672e-01, -4.3395e-01,
          3.9899e-01, -1.9573e-01,  1.3977e-01, -2.1519e-02,  3.7823e-01,
         -5.5250e-01, -1.1230e-01, -8.1443e-03,  2.9059e-01,  6.6817e-02,
          1.0465e-01, -8.6943e-02, -4.8983e-02, -2.6757e-01, -4.7038e-01,
          2.7469e-01,  6.9245e-02, -2.7967e-02, -1.9719e-01,  1.6749e-02,
         -2.9681e-01,  1.7838e-01,  5.8374e-02, -2.4806e-01,  8.5846e-02,
          3.5043e-01,  4.9157e-02, -1.6431e-01,  5.0012e-01, -1.8053e-01,
          3.1422e-01,  1.0671e-01,  3.1852e-02,  7.4278e-02,  2.7956e-01,
          8.0317e-02,  5.4780e-02, -3.0349e-01, -4.3215e-01,  3.2417e-01,
          4.0856e-01,  3.6192e-01,  1.3445e-01, -1.2933e-01,  1.1331e-01,
         -1.5755e-01,  3.5755e-01,  3.0463e-01, -9.8488e-02,  1.2032e-02,
          4.5581e-01,  3.7101e-01,  1.4270e-01, -4.3329e-01, -1.0869e-01,
          4.9849e-01,  5.4455e-

In [28]:
len(vocabset)

2188

BUILD MOR: POS + GRAMMATICAL CATEGORIES

In [ ]:
posset = set()

In [ ]:
for corpus,data in [(cookie_fnames,cookie_data),(sentence_fnames,sentence_data)]: 
    for i in range(len(corpus)):
        pos = set([pos for sent in data.tagged_sents(participant='PAR',by_files=True)[corpus[i]] for (tokenraw,pos,tokenstem,dependency) in sent])
        posset.update(pos)

In [ ]:
len(posset)

In [ ]:
# to concat to embedding tensor 
pos_dict = {i:pos for (i,pos) in enumerate(posset)}

CLEAN POS, VOCAB


note '' pos, '.' token: <EOS> 

In [34]:
#https://github.com/Blosc/bcolz
import bcolz

In [31]:
# import zipfile

In [32]:
# with zipfile.ZipFile('./PretrainedWordEmb/glove.42B.300d.zip', 'r') as zip_ref:
#     zip_ref.extractall('./PretrainedWordEmb/')

In [35]:
glove_path = './PretrainedWordEmb/'

In [52]:
with open(f'{glove_path}/glove.42B.300d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        break
    

In [57]:
len(line) - 1

300

In [58]:
words = []
idx = 0
word2idx = {}
vectors = bcolz.carray(np.zeros(1), rootdir=f'{glove_path}/glove.42B.300.dat', mode='w')

with open(f'{glove_path}/glove.42B.300d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)

In [61]:
vectors.shape

(575248201,)

In [60]:
vectors[1:].shape

(575248200,)

In [45]:
int(42E9)

42000000000

In [62]:
emptyvec = np.zeros((575248200,))

In [64]:
emptyvec.reshape((-1,300)).shape

(1917494, 300)

In [66]:
import pickle

In [65]:
vectors = bcolz.carray(vectors[1:].reshape((-1,300)), rootdir=f'{glove_path}/glove.42B.300.dat', mode='w')
vectors.flush()

In [67]:
pickle.dump(words, open(f'{glove_path}/glove.42B.300_words.pkl', 'wb'))
pickle.dump(word2idx, open(f'{glove_path}/glove.42B.300_idx.pkl', 'wb'))

In [70]:
vectors = bcolz.open(f'{glove_path}/glove.42B.300.dat')[:]

In [70]:
words = pickle.load(open(f'{glove_path}/glove.42B.300_words.pkl', 'rb'))
word2idx = pickle.load(open(f'{glove_path}/glove.42B.300_idx.pkl', 'rb'))

In [180]:
'UNK' in words

False

In [184]:
type(words)

list

In [186]:
words[1]

'the'

In [188]:
type(word2idx)

dict

In [189]:
words.append('UNK')

In [199]:
word2idx['þi']

1917493

In [201]:
words[-1]

'UNK'

In [202]:
words[-2]

'þi'

In [203]:
words[0]

','

In [204]:
len(vectors)

1917494

In [209]:
type(vectors)

numpy.ndarray

In [212]:
vectors.shape

(1917494, 300)

In [217]:
a = np.array([[1, 2], [3, 4]])

In [218]:
b = np.array([[5, 6]])

In [219]:
a.shape

(2, 2)

In [220]:
b.shape

(1, 2)

In [225]:
np.concatenate((a,b)).shape

(3, 2)

In [244]:
vectors = np.concatenate((vectors,np.reshape(average_vec,(1,300))))

In [245]:
len(word2idx)

1917495

In [251]:
word2idx[list(word2idx)[1917493]]

1917493

In [252]:
len(vectors)

1917495

In [253]:
word2idx['UNK'] = len(vectors) - 1

In [241]:
np.reshape(average_vec,(1,300)).shape

(1, 300)

In [227]:
vectors.shape

(1917494, 300)

In [200]:
len(words)

1917495

In [191]:
word2idx['UNK'] = len(word2idx)

In [194]:
word2idx['the']

1

In [195]:
type(vectors)

numpy.ndarray

In [215]:
average_vec.shape

(300,)

In [237]:
np.mean([v[0] for v in vectors])

-0.006771107619242646

In [238]:
np.mean([v[1] for v in vectors])

-0.0023817160208801304

In [243]:
np.reshape(average_vec,(1,300))[0,0]

-0.00677110761924225

In [182]:
average_vec.shape

(300,)

In [254]:
from nltk.tokenize.treebank import TreebankWordTokenizer

def get_glovembs(sample):
    
    def handle_contractions(x):
        tokenizer = TreebankWordTokenizer()
        x = tokenizer.tokenize(x)
        x = ' '.join(x)
        return x
    
    # preprocess acc to pretrained embeddings 
    # lower
    # replace _, -
    # rm contractions
    # rm nonascii chars
    
    sample = [s for s in sample]
    
    preprocessample = [handle_contractions(t).split()[0] for t in [token.lower().replace('_','-') for token in sample]]
    
    preprocessample = [''.join([i if ord(i) < 128 else '' for i in w]) for w in preprocessample]
    # {w: vectors[word2idx[w]] for w in words}
    return {sample[i]: torch.tensor(vectors[word2idx[preprocessample[i]]]) if preprocessample[i] in words else torch.tensor(vectors[word2idx['UNK']]) for i in range(len(sample))}

In [255]:
addb_vocab_embs = get_glovembs(vocabset)

In [257]:
len(addb_vocab_embs)

2188

In [272]:
addb_vocab_embs['UNK']

KeyError: 'UNK'

In [277]:
addb_vocab_embs['CLITIC'] = vectors[word2idx['UNK']]

In [278]:
np.array(addb_vocab_embs['CLITIC']) == (vectors[word2idx['clitic']])

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [276]:
np.array(addb_vocab_embs['CLITIC']) == (vectors[word2idx['clitic']])

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [279]:
pickle.dump(addb_vocab_embs, open(f'{glove_path}/addb.vocab.glove.42B.300_words.pkl', 'wb'))

In [280]:
addbvocab = pickle.load(open(f'{glove_path}/addb.vocab.glove.42B.300_words.pkl', 'rb'))

In [282]:
len(addbvocab)

2188

In [283]:
list(addbvocab) == list(addb_vocab_embs)

True

In [84]:
vocabset_proc = [t for t in vocabset]

In [86]:
vocabset_proc = [t.lower() for t in vocabset_proc]

In [94]:
vocabset_proc = [t.replace('_','-') for t in vocabset_proc]

In [129]:
vocabset_proc = [handle_contractions(x).split()[0] for x in vocabset_proc]

In [151]:
vocabset_proc = [''.join([i if ord(i) < 128 else '' for i in u]) for u in vocabset_proc]

In [152]:
len(vocabset_proc)

2188

In [119]:
from nltk.tokenize.treebank import TreebankWordTokenizer


In [120]:
#https://www.kaggle.com/christofhenkel/how-to-preprocessing-for-glove-part1-eda
def handle_contractions(x):
    x = tokenizer.tokenize(x)
    x = ' '.join(x)
    return x

In [153]:
unks = [t for t in vocabset_proc if t not in words]

In [154]:
len(unks)

26

In [267]:
'not-a-word' in words

False

In [155]:
unks

['cookeiejar',
 "i-don't-know",
 'cappdf',
 '//',
 '',
 '//',
 'alrightie',
 '..',
 'oh-boy',
 'hmhunh',
 'sewickly',
 'tazmania-dutch',
 'doctor-dick',
 'how-about',
 'god-bless-you',
 '',
 'oh-dear',
 'how-come',
 'i-mean',
 'what-about',
 'joyce-kilmer',
 'the-window-of-the-xxx',
 'dave-branton',
 'lots-of',
 'a-lot-of',
 'lee-a']

In [ ]:
# ignore CLITIC

In [135]:
'a-lot' in words

True

In [137]:
import string

In [138]:
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [140]:
printable = set(string.printable)

In [136]:
[u.strip() for u in unks]

['∬okay∬',
 '∬open',
 'cookeiejar',
 "i-don't-know",
 'cappdf',
 '//',
 '‡',
 '//',
 'alrightie',
 '..',
 'oh-boy',
 'hmhunh',
 'sewickly',
 'tazmania-dutch',
 '∬sake∬',
 'doctor-dick',
 'drawer∬',
 'how-about',
 'god-bless-you',
 'oh-dear',
 'how-come',
 '∬no∬',
 'i-mean',
 'what-about',
 'joyce-kilmer',
 'the-window-of-the-xxx',
 'dave-branton',
 'lots-of',
 'a-lot-of',
 'lee-a']

In [133]:
unks

['∬okay∬',
 '∬open',
 'cookeiejar',
 "i-don't-know",
 'cappdf',
 '//',
 '‡',
 '//',
 'alrightie',
 '..',
 'oh-boy',
 'hmhunh',
 'sewickly',
 'tazmania-dutch',
 '∬sake∬',
 'doctor-dick',
 'drawer∬',
 'how-about',
 'god-bless-you',
 'oh-dear',
 'how-come',
 '∬no∬',
 'i-mean',
 'what-about',
 'joyce-kilmer',
 'the-window-of-the-xxx',
 'dave-branton',
 'lots-of',
 'a-lot-of',
 'lee-a']

In [91]:
'aint' in words

True

In [183]:
unks

['cookeiejar',
 "i-don't-know",
 'cappdf',
 '//',
 '',
 '//',
 'alrightie',
 '..',
 'oh-boy',
 'hmhunh',
 'sewickly',
 'tazmania-dutch',
 'doctor-dick',
 'how-about',
 'god-bless-you',
 '',
 'oh-dear',
 'how-come',
 'i-mean',
 'what-about',
 'joyce-kilmer',
 'the-window-of-the-xxx',
 'dave-branton',
 'lots-of',
 'a-lot-of',
 'lee-a']

In [ ]:
glove = {w: vectors[word2idx[w]] for w in words}